In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

今回のコンペでは、与えられた気象データから、carbon monoxide（一酸化炭素）、benzene（ベンゼン）、nitrogen oxides（窒素酸化物）の3つの量を予測する。

In [ ]:
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt 

学習用、テスト用、提出用データをそれぞれ読み込む

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")

## データの確認

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
train.info()
test.info()

## データの前処理


特徴量の一つ、date_timeをstr型（文字列型）からdatetime型に変換する

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], format = "%Y-%m-%d %H:%M:%S")
test['date_time'] = pd.to_datetime(test['date_time'], format= "%Y-%m-%d %H:%M:%S")
target_name = train.columns[-3:].values # 目的変数名を保存しておく
target_name

「時間」に関係する、さまざまな特徴量を追加する

In [ ]:
def makeFeatures(df):
    df["month"] = df["date_time"].dt.month # 月
    df["day_of_week"] = df["date_time"].dt.dayofweek # 曜日（月曜始まり）
    df["day_of_year"] = df["date_time"].dt.dayofyear # 1月1日から数えて何日目か
    df["hour"] = df["date_time"].dt.hour # 時
    df["quarter"] = df["date_time"].dt.quarter # 四半期
    df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype("int") # 1月1日から数えて何週目か
    df["is_sprint"] = df["month"].isin([3, 4, 5]) # 春
    df["is_summer"] = df["month"].isin([6, 7, 8]) # 夏
    df["is_autumn"] = df["month"].isin([9, 10, 11]) # 秋
    df["is_winter"] = df["month"].isin([1, 2, 12]) # 冬
    df["working_hours"] =  df["hour"].isin(np.arange(8, 19, 1)).astype("int") # 勤務時間(8時～19時)
    df["is_weekend"] = (train["date_time"].dt.dayofweek >= 5).astype("int") # 週末
    return df

In [ ]:
train = makeFeatures(train)
test = makeFeatures(test)

datetime型の特徴量を学習に用いるため、int型（整数型）に変換する

In [ ]:
train['date_time'] = train['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9
test['date_time'] = test['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9

学習用データの中身は3つの項目（target_carbon_monoxide, target_benzene, target_nitrogen_oxides）が目的変数となっていて、のこりが特徴量（説明変数）となっている。  
また、テスト用データの中身は学習用データから3つの目的変数を除いた構造になっている。  
以下のプログラムでは、学習用データから特徴量だけを取り出したデータ（X）と目的変数だけ取り出したデータ（target）を作成している。

In [ ]:
columns = test.columns
X = train[columns].values
X_test = test[columns].values
target0 = train['target_carbon_monoxide'].values.reshape(-1,1)
target1 = train['target_benzene'].values.reshape(-1,1)
target2 = train['target_nitrogen_oxides'].values.reshape(-1,1)
target = np.concatenate([target0, target1, target2], 1)

## 前処理後のデータ確認

In [ ]:
X_df = pd.DataFrame(data=X, columns=columns)
X_df.head()

In [ ]:
X_test_df = pd.DataFrame(data=X_test, columns=columns)
X_test_df.head()

In [ ]:
# 学習用データの内容を表示する
print(f"学習用データ：{train.shape[0]}行{train.shape[1]}列")
print(f"学習用データ数：{train.shape[0]}個")
print()
print(f"特徴量のデータ：{X.shape[0]}行{X.shape[1]}列")
print(f"特徴量の数：{X.shape[1]}個")
print(f"特徴量の名前：{columns.values}")
print()
print(f"目的変数のデータ：{target.shape[0]}行{target.shape[1]}列")
print(f"目的変数の数：{target.shape[1]}個")
print(f"目的変数の名前：{train.columns[-3:].values}")

In [ ]:
# テスト用データの内容を表示する
print(f"テスト用データ：{test.shape[0]}行{test.shape[1]}列")
print(f"テスト用データ数：{test.shape[0]}個")
print()
print(f"特徴データ：{X_test.shape[0]}行{X_test.shape[1]}列")
print(f"特徴量の数：{X_test.shape[1]}個")
print(f"特徴量の種類：{columns.values}")

In [ ]:
# 内容を表示する
print(f"提出用データ：{sample_submission.shape[0]}行{sample_submission.shape[1]}列")
print(f"提出用データ数：{sample_submission.shape[0]}個")

## ランダムフォレストによる学習

ランダムフォレストの構築

In [ ]:
# ニューラルネットワーク
#mlp = MLPRegressor(hidden_layer_sizes=(256, 16), max_iter=500, random_state=1, verbose=False)
mlp = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=1, verbose=0, min_samples_split=6)

ランダムフォレストのハイパーパラメータを決めるため、グリッドサーチを行い、最適なパラメータを選択する。また、5分割交差検証を行うことで、過学習を防ぎ正しく精度を評価する。

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))
    rank = results.cv_results_['rank_test_score']
    arg = np.argsort(rank)
    rank = np.sort(rank)
    means = results.cv_results_['mean_test_score'][arg]
    stds = results.cv_results_['std_test_score'][arg]
    params = np.array(results.cv_results_['params'])[arg]

    for r, mean, std, param in zip(rank, means, stds, params):
        print('No{} : {} (+/-{}) for {}'.format(r, round(mean, 3), round(std * 2, 3), param))

def gridSearch(model, X_train, y_train):
    paramters = {
            "n_estimators": [i for i in range(10, 100, 5)],
            "max_depth":[i for i in range(5, 10)]
    }
    
    grid = GridSearchCV(
        estimator = model,
        param_grid = paramters,
        scoring = 'neg_root_mean_squared_error',
        cv = 5,
        verbose = 0
    )
    grid.fit(X_train, y_train)
    print_results(grid)

In [ ]:
#gridSearch(mlp, X, target)

実際にランダムフォレストで学習する。グリッドサーチを行った時と同様に5分割交差検証を行う。

In [ ]:
# 評価する指標
score_funcs = [
    'neg_root_mean_squared_error'
]

score = cross_validate(mlp, X, target, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1, verbose=3)
print(np.mean(score['test_score']))

In [ ]:
mlp.fit(X, target)

## 学習結果

学習したランダムフォレストモデルを使用して、テストデータから予測する。

In [ ]:
pred = mlp.predict(X_test)
sample_submission[sample_submission.columns[1:]] = pred
sample_submission.head()

予測した結果をファイルに保存する

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

テスト用データは目的変数が公開されていないため、学習用データを使って予測結果をグラフに表示する。

In [ ]:
train_pred = mlp.predict(X)
plt.figure(figsize=(16, 20))
for i, name in enumerate(target_name):
    plt.subplot(3, 1, i + 1)
    plt.plot(target[:500, i])
    plt.plot(train_pred[:500, i])
    plt.title(name)